### Create training set

In [44]:
import numpy as np
from random import shuffle

#add binary string a, and b to create c.
def add(a, b):
    l = max(len(a), len(b));
    a = int(a, 2);
    b = int(b, 2);
    c = bin(a+b)[2:].zfill(l);
    return c;

def str2np(a):
    result = [int(x) for x in a]
    return result;

def lstofstr2tensor(lstofstr):
    tensor = [str2np(string) for string in lstofstr]
    return tensor

def merge_str2list(str1, str2):
    result = [[int(str1[i]), int(str2[i])] for i in xrange(len(str1))]
    return result
N_bit = 15;
#To prevent overloading bit on result, we only generate N_bit-1 string
formatting_string = ('{0:0'+str(N_bit)+'b}')
input_a = [formatting_string.format(i) for i in xrange(2**(N_bit-1))];
input_b = [formatting_string.format(i) for i in xrange(2**(N_bit-1))];
shuffle(input_a);
shuffle(input_b);

N_TOTAL = 2000;

input_a = input_a[:N_TOTAL];
input_b = input_b[:N_TOTAL];
target_c = [add(input_a[i], input_b[i]) for i in xrange(N_TOTAL)];

#Need to reverse the binary string, since we do the calculation from right to left
input_a = [a[::-1] for a in input_a];
input_b = [b[::-1] for b in input_b];
target_c = [c[::-1] for c in target_c];

#Merge string a and b to one input
input_ab = [merge_str2list(input_a[i], input_b[i]) for i in xrange(N_TOTAL)];

target_c = lstofstr2tensor(target_c);

#Convert to numpy array
input_ab = np.asarray(input_ab)
target_c = np.asarray(target_c)

#Split to training and testing dataset
N_TRAIN = 1500;
test_input_ab = input_ab[N_TRAIN:]
test_target_c = target_c[N_TRAIN:] 

train_input_ab = input_ab[:N_TRAIN]
train_target_c = target_c[:N_TRAIN] 


### Use Tensorflow for LSTM
#### First we define the computational graph

In [91]:
import tensorflow as tf
tf.reset_default_graph()

#Placeholder for input and target, [Batch Size, Sequence Length, Input Dimension]
data = tf.placeholder(tf.float32, [None, N_bit, 2])
target = tf.placeholder(tf.float32, [None, N_bit])

num_hidden = 5
time_steps = N_bit

lstmCell = tf.nn.rnn_cell.BasicLSTMCell(num_hidden, forget_bias=1.0, state_is_tuple=True)

sequence_length 
#unroll the LSTM network
outputs, states = tf.nn.dynamic_rnn(cell=lstmCell, inputs=data, dtype=tf.float32)

# Define weights
out_size = 1;

# Weights for each timestep output, using the same weight
weight = tf.Variable(tf.truncated_normal([num_hidden, out_size], stddev=0.1))
bias = tf.Variable(tf.constant(0.1, shape=[out_size]))

outputs = tf.reshape(outputs, [-1, num_hidden])
prediction = tf.sigmoid(tf.matmul(outputs, weight) + bias)
prediction = tf.reshape(prediction, [-1, time_steps, out_size])
prediction = tf.squeeze(prediction)

cross_entropy = -tf.reduce_mean(target * tf.log(prediction) + (1-target)*tf.log(1-prediction))
#cross_entropy = tf.reduce_mean(cross_entropy)

#cross_entropy = -tf.reduce_sum(target * tf.log(prediction))

optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

final_prediction = prediction>0.5;
mistakes = tf.not_equal(target>0.5, final_prediction) #careful when target is still float32
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))


#### Then we execute the graph

In [101]:
init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

BATCH_SIZE = 100
no_of_batches = int(len(train_input_ab)/BATCH_SIZE)
epoch = 200
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out = train_input_ab[ptr:ptr+BATCH_SIZE], train_target_c[ptr:ptr+BATCH_SIZE]
        ptr+=BATCH_SIZE
        sess.run(minimize,{data: inp, target: out})
        #print(sess.run(cross_entropy,{data: inp, target: out}))
    print "Epoch - ",str(i)
incorrect = sess.run(error,{data: test_input_ab, target: test_target_c})
print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))
prediction = sess.run(final_prediction,{data: test_input_ab, target: test_target_c})
prediction.astype(int)

#sess.close()

Epoch -  0
Epoch -  1
Epoch -  2
Epoch -  3
Epoch -  4
Epoch -  5
Epoch -  6
Epoch -  7
Epoch -  8
Epoch -  9
Epoch -  10
Epoch -  11
Epoch -  12
Epoch -  13
Epoch -  14
Epoch -  15
Epoch -  16
Epoch -  17
Epoch -  18
Epoch -  19
Epoch -  20
Epoch -  21
Epoch -  22
Epoch -  23
Epoch -  24
Epoch -  25
Epoch -  26
Epoch -  27
Epoch -  28
Epoch -  29
Epoch -  30
Epoch -  31
Epoch -  32
Epoch -  33
Epoch -  34
Epoch -  35
Epoch -  36
Epoch -  37
Epoch -  38
Epoch -  39
Epoch -  40
Epoch -  41
Epoch -  42
Epoch -  43
Epoch -  44
Epoch -  45
Epoch -  46
Epoch -  47
Epoch -  48
Epoch -  49
Epoch -  50
Epoch -  51
Epoch -  52
Epoch -  53
Epoch -  54
Epoch -  55
Epoch -  56
Epoch -  57
Epoch -  58
Epoch -  59
Epoch -  60
Epoch -  61
Epoch -  62
Epoch -  63
Epoch -  64
Epoch -  65
Epoch -  66
Epoch -  67
Epoch -  68
Epoch -  69
Epoch -  70
Epoch -  71
Epoch -  72
Epoch -  73
Epoch -  74
Epoch -  75
Epoch -  76
Epoch -  77
Epoch -  78
Epoch -  79
Epoch -  80
Epoch -  81
Epoch -  82
Epoch -  83
Ep

array([[1, 1, 0, ..., 0, 1, 0],
       [1, 1, 1, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 1],
       ..., 
       [1, 0, 0, ..., 0, 1, 0],
       [1, 0, 1, ..., 0, 1, 1],
       [0, 1, 1, ..., 0, 0, 1]])

In [103]:
#Some prediction examples:
prediction = sess.run(final_prediction,{data: test_input_ab, target: test_target_c})
print test_input_ab[59]
print test_target_c[59]
print prediction[59]

[[0 0]
 [0 1]
 [0 1]
 [1 1]
 [0 1]
 [0 1]
 [1 1]
 [0 1]
 [1 0]
 [1 0]
 [0 0]
 [0 0]
 [0 1]
 [1 1]
 [0 0]]
[0 1 1 0 0 0 1 0 0 0 1 0 1 0 1]
[False  True  True False False False  True False False False  True False
  True False  True]
